You have to work on the files:
*  [Books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz)
*  [Book ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz)
*  [Users](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz)
*  [Goodbooks books](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz)
*  [Goodbooks ratings](https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz)

### Notes

1.    It is mandatory to use GitHub for developing the project.
1.    The project must be a jupyter notebook.
1.    There is no restriction on the libraries that can be used, nor on the Python version.
1.    To read those files, you need to use the `encoding = 'latin-1'` option.
1.    All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com), otherwise no  answer will be given.

In [1]:
# Importiamo i pacchetti
import pandas as pd, re, numpy as np

## IMPORTIAMO I DATAFRAMES

#### "BOOKS"

In [2]:
books = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Books.csv.gz', sep = ';', low_memory = False, encoding = 'latin-1')
books.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


##### PULIZIA CODICI ISBN - dataframe BOOKS

In [3]:
def pulisci_ISBN(ISBN_passato):
    if ISBN_passato[-1] == 'X' or ISBN_passato[-1] == 'x':
        return ''.join(re.findall('(\d*)', str(ISBN_passato))) + 'X'
    else:
        return ''.join(re.findall('(\d*)', str(ISBN_passato))) 

books['Cleaned_ISBN'] = books['ISBN'].apply(pulisci_ISBN)

#### "BOOK RATINGS"

In [4]:
book_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Book-Ratings.csv.gz', sep = ';', encoding = 'latin-1')
book_ratings.head(3)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


###### ANTEFATTO: PULIZIA CODICI ISBN DEL DATAFRAME "BOOK RATINGS"

In [5]:
# I codici ISBN avranno il loro peso all'interno del progetto.
# Ecco una versione che contiene SOLO i codici ISBN corretti del dataframe "BOOK RATINGS"

# --------------------------------------------------------------------------------------------------------------- #

# Ripuliamo gli ISBN perché siano solo numerici - o finiscano in 'X', e vediamo adesso


def pulisci_ISBN(ISBN_passato):
    if ISBN_passato[-1] == 'X' or ISBN_passato[-1] == 'x':
        return ''.join(re.findall('(\d*)', str(ISBN_passato))) + 'X'
    else:
        return ''.join(re.findall('(\d*)', str(ISBN_passato))) 

book_ratings['Cleaned_ISBN'] = book_ratings['ISBN'].apply(pulisci_ISBN)

# Vediamo come si presenta il conteggio della lunghezza degli ISBN

def lunghezza_stringa(stringa):
    return len(str(stringa))

book_ratings['len_ISBN'] = book_ratings['ISBN'].apply(lunghezza_stringa)
book_ratings['len_Cleaned_ISBN'] = book_ratings['Cleaned_ISBN'].apply(lunghezza_stringa)

# Questi sono i codici ISBN10 già perfetti

ISBN10_book_ratings = book_ratings[book_ratings['len_Cleaned_ISBN']==10]

# Qui creiamo i codici ISBN13 autentici

def trova_ISBN13_corretti(ISBN_in_esame):
    if len(ISBN_in_esame)==13 and (ISBN_in_esame[:3]=='978' or ISBN_in_esame[:3]=='979'):
        return ISBN_in_esame
    else:
        return 0
    
book_ratings['Cleaned_ISBN'] = book_ratings['Cleaned_ISBN'].apply(trova_ISBN13_corretti)

ISBN13_book_ratings = book_ratings[book_ratings['Cleaned_ISBN']!=0]

# Questo è il dataframe definitivo

revised_book_ratings = pd.concat([ISBN10_book_ratings,ISBN13_book_ratings])[['User-ID', 'Cleaned_ISBN', 'Book-Rating']]

revised_book_ratings.head()

,User-ID,Cleaned_ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


#### "USERS"

In [6]:
users = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/Users.csv.gz', sep = ';', encoding = 'latin-1')
users.head(3)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN


#### "GOODBOOKS"

In [7]:
goodbooks_books = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks.csv.gz', sep = ',', encoding = 'latin-1')
goodbooks_books.head(3)

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPrÃ©",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...


###### PULIZIA DEI CODICI ISBN - DATAFRAME "GOODBOOKS"

In [8]:
# Con queste funzioni risolviamo molti dei problemi legati alla lettura dei codici ISBN

def repair_ISBN10(corrupted_ISBN):
    if len(str(corrupted_ISBN)) < 10:
        for time in range(10 - len(str(corrupted_ISBN))):
            corrupted_ISBN = '0' + str(corrupted_ISBN)
    if str(corrupted_ISBN)[-1] == 'x':
        corrupted_ISBN = str(corrupted_ISBN)[:-1] + 'X'
    return corrupted_ISBN

def repair_ISBN13(corrupted_ISBN):
    corrupted_ISBN = str(corrupted_ISBN)[:13]
    if str(corrupted_ISBN)[-1] == 'x':
        corrupted_ISBN = str(corrupted_ISBN)[:-1] + 'X'
    return corrupted_ISBN

goodbooks_books['Cleaned_isbn'] = goodbooks_books['isbn'].apply(repair_ISBN10)
goodbooks_books['Cleaned_isbn13'] = goodbooks_books['isbn13'].apply(repair_ISBN13)

#### "GOODBOOKS RATINGS"

In [9]:
goodbooks_ratings = pd.read_csv('https://github.com/gdv/foundationsCS/raw/master/progetti/2021/goodbooks-ratings.csv.gz', encoding = 'latin-1')
goodbooks_ratings.head(3)

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5


### 1. Normalize the location field of *Users* dataset, splitting into city, region, country.

In [10]:
# Creiamo una funzione che ripulisca la colonna 'Location'

def location_cleaner(location_string):

    # trasmormiamo la stringa in lista, eliminando spazi inutili, le maiuscole e i campi vuoti
    
    new_name_list = []
    location_string = location_string.split(',')
    for name in location_string:
        name = re.sub(r'^([\s])','', name)
        name = re.sub(r'([\s])$','', name)
        name = name.lower()
        if re.match(r'([\s]+)', name) or name == '':
            name = 1
        else:
            new_name_list.append(name)
            
    # elimina i campi ripetuti, avendo cura di "spingere gli originali in fondo"
    
    new_list = []
    new_name_list = new_name_list[::-1]
        
    for name in new_name_list:
        if name in new_list:
            name = 1
        else:
            new_list.append(name)
        
    new_list = new_list[::-1]
    
# normalizziamo la lunghezza delle liste a 3 per ciascuna, eliminando all'occorrenza gli elementi in eccesso

    # accediamo come (name_list)
        
    returning_list = [None, None, None]    
     
    returning_list += new_list
        
    return returning_list[-3:]

In [11]:
# Creiamo una colonna con i valori di 'Location' ripuliti dalla funzione:
users['Cleaned_location'] = users['Location'].apply(location_cleaner)
# --------------------------------------------------------------------------------------------------------------- #
# Distribuiamo i valori delle liste sulle tre colonne desiderate

def distribute_cities(list_location):
    return list_location[0]
def distribute_regions(list_location):
    return list_location[1]
def distribute_countries(list_location):
    return list_location[2]

users['City'] = users['Cleaned_location'].apply(distribute_cities)
users['Region'] = users['Cleaned_location'].apply(distribute_regions)
users['Country'] = users['Cleaned_location'].apply(distribute_countries)
# --------------------------------------------------------------------------------------------------------------- #
# Eliminiamo tutte le colonne che non ci servono
users = users[['User-ID','City', 'Region', 'Country', 'Age']]
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato
users

,User-ID,City,Region,Country,Age
0,1,nyc,new york,usa,NaN
1,2,stockton,california,usa,18.0
2,3,moscow,yukon territory,russia,NaN
3,4,porto,v.n.gaia,portugal,17.0
4,5,farnborough,hants,united kingdom,NaN
...,...,...,...,...,...
278853,278854,portland,oregon,usa,NaN
278854,278855,tacoma,washington,united kingdom,50.0
278855,278856,brampton,ontario,canada,NaN
278856,278857,knoxville,tennessee,usa,NaN


### 2. For each book in the *Books* dataset, compute its average rating.

In [12]:
# Prima di tutto, elimino i rating "0", dando per scontato che sia un valore per sostituzione dei valori Null

counting_book_ratings = revised_book_ratings[revised_book_ratings['Book-Rating']!=0]

In [13]:
# Facciamo una LEFT JOIN tra libri e ratings (se presenti)
books_and_ratings = pd.merge(books, counting_book_ratings, on = 'Cleaned_ISBN', how = 'left') 
# --------------------------------------------------------------------------------------------------------------- #
# Facciamo la media dei voti dei libri (per titolo)
rating_for_books = books_and_ratings.groupby('ISBN', as_index = False)['Book-Rating'].mean() 
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo solo le colonne rilevanti
rating_for_books = rating_for_books[['ISBN', 'Book-Rating']] 
# --------------------------------------------------------------------------------------------------------------- #
# Rinominiamo la colonna interessata
rating_for_books = rating_for_books.rename(columns={'Book-Rating':'AVG_Book_rating'}) 
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato 
rating_for_books

,ISBN,AVG_Book_rating
0,0000913154,8.0
1,0001010565,NaN
2,0001046438,9.0
3,0001046713,NaN
4,000104687X,6.0
...,...,...
271354,B000234N76,NaN
271355,B000234NC6,NaN
271356,B00029DGGO,NaN
271357,B0002JV9PY,NaN


### 3. For each book in the *GoodBooks* dataset, compute its average rating.

In [14]:
# Calcoliamo il rating medio creando una colonna apposita: facciamo la media ponderata dei 5 voti di rating 
goodbooks_books['Calculated_average_rating'] = (goodbooks_books['ratings_1']*1.0 + goodbooks_books['ratings_2']*2.0 + goodbooks_books['ratings_3']*3.0 + goodbooks_books['ratings_4']*4.0 + goodbooks_books['ratings_5']*5.0)/goodbooks_books['work_ratings_count'] 
# --------------------------------------------------------------------------------------------------------------- #
# Lasciamo solo le colonne rilevanti
avg_rating_for_book = goodbooks_books[['book_id', 'title', 'average_rating', 'Calculated_average_rating']] 
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato
avg_rating_for_book 

,book_id,title,average_rating,Calculated_average_rating
0,1,"The Hunger Games (The Hunger Games, #1)",4.34,4.341984
1,2,Harry Potter and the Sorcerer's Stone (Harry P...,4.44,4.443047
2,3,"Twilight (Twilight, #1)",3.57,3.571067
3,4,To Kill a Mockingbird,4.25,4.254818
4,5,The Great Gatsby,3.89,3.887376
...,...,...,...,...
94,95,The Picture of Dorian Gray,4.06,4.055317
95,96,"Fifty Shades Freed (Fifty Shades, #3)",3.88,3.879526
96,97,Dracula,3.98,3.976401
97,98,"The Girl Who Played with Fire (Millennium, #2)",4.22,4.220469


### 4. Merge together all rows sharing the same book title, author and publisher. We will call the resulting datset `merged books`. The books that have not been merged together will not appear in `merged books`.

In [15]:
# RIPRISTINIAMO 3 RIGHE DEL DATAFRAME "BOOKS" CHE HANNO PROBLEMI CON LA DIVISIONE PER ';'

# --------------------------------------------------------------------------------------------------------------- #

# Creiamo un dataframe per le righe separate male
messed_books = books[books['Image-URL-L'].isnull()]
# --------------------------------------------------------------------------------------------------------------- #
# E poi creiamo un dataframe con tutti i libri restanti
unmessed_books = books[~books['Image-URL-L'].isnull()]
# --------------------------------------------------------------------------------------------------------------- #
# Slittiamo gli altri valori di una posizione
for j in range (len(messed_books)):
    for i in range(5):    
        messed_books.iloc[[j],[-(i+1)]] = messed_books.iloc[[j],[-(i+2)]] 
# --------------------------------------------------------------------------------------------------------------- #
# Forziamo la divisione
def split_and_fill(wrong_title): 
    return wrong_title.split(';')

messed_books['Splitted-Book-Title'] = messed_books['Book-Title'].apply(split_and_fill)
# --------------------------------------------------------------------------------------------------------------- #
# Attribuiamo il titolo corretto
def distribute_names(list_title_authors): 
    return list_title_authors[0]
# --------------------------------------------------------------------------------------------------------------- #
# Attribuiamo l'autore corretto
def distribute_authors(list_title_authors): 
    return list_title_authors[1]
# --------------------------------------------------------------------------------------------------------------- #
# Distribuiamo il contenuto nelle colonne opportune
messed_books['Book-Title'] = messed_books['Splitted-Book-Title'].apply(distribute_names)
messed_books['Book-Author'] = messed_books['Splitted-Book-Title'].apply(distribute_authors)
# --------------------------------------------------------------------------------------------------------------- #
# Eliminiamo la colonna in eccesso
messed_books = messed_books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M', 'Image-URL-L']]
# --------------------------------------------------------------------------------------------------------------- #
# Facciamo in modo che il dataframe Books sia la somma dei due
books = pd.concat([unmessed_books, messed_books])

C:\Users\CRD\AppData\Local\Temp\ipykernel_12004\3251329241.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messed_books.iloc[[j],[-(i+1)]] = messed_books.iloc[[j],[-(i+2)]]
C:\Users\CRD\AppData\Local\Temp\ipykernel_12004\3251329241.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messed_books['Splitted-Book-Title'] = messed_books['Book-Title'].apply(split_and_fill)
C:\Users\CRD\AppData\Local\Temp\ipykernel_12004\3251329241.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [16]:
# SOLUZIONI QUESITO N. 4

# Prima creiamo un dataframe senza "doppioni" in quei tre campi
grouped_books = books.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index = False)['ISBN'].count() 
# --------------------------------------------------------------------------------------------------------------- #
# Teniamo solo le righe con count > 1 eliminando le altre
merged_books = grouped_books[grouped_books['ISBN']>1][['Book-Title', 'Book-Author', 'Publisher']] 
# --------------------------------------------------------------------------------------------------------------- #
# Ripristiniamo un indice implicito ordinato
merged_books = merged_books.reset_index(drop=True) 
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato
merged_books 

,Book-Title,Book-Author,Publisher
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books
3,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books
4,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group
...,...,...,...
4720,Zia,Scott O'Dell,Laurel-Leaf Books
4721,Zia,Scott O'Dell,Yearling Books
4722,Zimmermann Telegram,Barbara Tuchman,Ballantine Books
4723,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics


### 5. For each book in `merged books` compute its average rating.

The average is computed considering all books in `books` that have been merged.

In [17]:
# Facciamo il merge di books e book_ratings
rating_for_title = pd.merge(books, counting_book_ratings, left_on = 'ISBN', right_on = 'Cleaned_ISBN')[['Book-Title','Book-Author','Publisher','Book-Rating']]
# --------------------------------------------------------------------------------------------------------------- #
# Adesso raggruppiamo per la triade di merged_books, ma ricaviamo le MEDIE invece dei conteggi:
mean_rated_books = rating_for_title.groupby(['Book-Title', 'Book-Author', 'Publisher'], as_index = False)['Book-Rating'].mean() 
# --------------------------------------------------------------------------------------------------------------- #
# Facciamo la LEFT JOIN tra merged_books e i voti medi associati ad ogni terzina:
average_rating_for_merged = pd.merge(merged_books, mean_rated_books, on = ['Book-Title', 'Book-Author', 'Publisher'], how = 'left')
# --------------------------------------------------------------------------------------------------------------- #
# Rinomino l'ultima colonna:
average_rating_for_merged.rename(columns = {'Book-Rating': 'Average-Rating'}, inplace = True)
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato
average_rating_for_merged

,Book-Title,Book-Author,Publisher,Average-Rating
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,6.000000
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,8.500000
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,7.000000
3,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,7.666667
4,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,NaN
...,...,...,...,...
4720,Zia,Scott O'Dell,Laurel-Leaf Books,6.800000
4721,Zia,Scott O'Dell,Yearling Books,6.000000
4722,Zimmermann Telegram,Barbara Tuchman,Ballantine Books,8.000000
4723,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.000000


### 6. For each book in `merged books` compute the minimum and maximum of the average ratings over all corresponding books in the `books` dataset.

Hence for each book in `merged books` we will have exactly two values (a minimum and a maximum)

In [18]:
# Nel dataframe counting_book_ratings, calcolo il voto medio per ogni ISBN ripulito
mean_rating_ISBN = counting_book_ratings.groupby('Cleaned_ISBN', as_index = False)['Book-Rating'].mean()
mean_rating_ISBN.head()
# --------------------------------------------------------------------------------------------------------------- #
# Riunisco le terzine, tenendo anche la colonna "Cleaned_ISBN"
ISBN_grouped_books = books.groupby(['Book-Title','Book-Author','Publisher','Cleaned_ISBN'], as_index = False)['ISBN'].count()[['Book-Title','Book-Author','Publisher','Cleaned_ISBN']]
# --------------------------------------------------------------------------------------------------------------- #
# Elimino tutti i voti che non c'entrano facendo una INNER JOIN con merged_books
cropped_merged_books = pd.merge(merged_books, ISBN_grouped_books, on = ['Book-Title','Book-Author','Publisher'])[['Book-Title','Book-Author','Publisher','Cleaned_ISBN']]
# --------------------------------------------------------------------------------------------------------------- #
# Faccio il merge tra le due tabelle (mean_rating_ISBN, cropped_merged_books), così aggiungo anche i voti medi per ISBN
avg_rating_for_merged_books = pd.merge(mean_rating_ISBN, cropped_merged_books, on = 'Cleaned_ISBN')[['Book-Title','Book-Author','Publisher','Cleaned_ISBN','Book-Rating']]
# --------------------------------------------------------------------------------------------------------------- #
# Ora per ogni ISBN che ha almeno un voto troviamo il minimo e il massimo
min_for_merged = avg_rating_for_merged_books.loc[avg_rating_for_merged_books.groupby(['Book-Title','Book-Author','Publisher'])['Book-Rating'].idxmin()]
max_for_merged = avg_rating_for_merged_books.loc[avg_rating_for_merged_books.groupby(['Book-Title','Book-Author','Publisher'])['Book-Rating'].idxmax()]
# --------------------------------------------------------------------------------------------------------------- #
# Unisco gli ultimi 2 dataframe e rinomino le colonne
avg_extremes_for_merged = pd.merge(min_for_merged,max_for_merged,on = ['Book-Title','Book-Author','Publisher'])
avg_extremes_for_merged.rename(columns = {'Book-Rating_x':'Min_AVG_rating', 'Book-Rating_y':'Max_AVG_rating'}, inplace = True)
# --------------------------------------------------------------------------------------------------------------- #
# Tramite una LEFT JOIN, andiamo ad aggiungere tutti i libri di merged_books che non hanno ricevuto un voto
# Intanto rimuoviamo le colonne superflue
min_max_avg_rating = pd.merge(merged_books, avg_extremes_for_merged, on =['Book-Title','Book-Author','Publisher'], how='left')[['Book-Title','Book-Author','Publisher','Min_AVG_rating','Max_AVG_rating']]
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato
min_max_avg_rating

,Book-Title,Book-Author,Publisher,Min_AVG_rating,Max_AVG_rating
0,!%@ (A Nutshell handbook),Donnalyn Frey,O'Reilly,6.000000,6.00
1,'A Hell of a Place to Lose a Cow': An American...,Tim Brookes,National Geographic,8.500000,8.50
2,"10,000 dreams interpreted: A dictionary of dreams",Gustavus Hindman Miller,Barnes &amp; Nobles Books,6.666667,7.25
3,101 Famous Poems,Roy J. Cook,McGraw-Hill/Contemporary Books,6.500000,10.00
4,15 Houseplants Even You Can't Kill,Joe Elder,Berkley Pub Group,NaN,NaN
...,...,...,...,...,...
4720,Zia,Scott O'Dell,Laurel-Leaf Books,6.000000,10.00
4721,Zia,Scott O'Dell,Yearling Books,6.000000,6.00
4722,Zimmermann Telegram,Barbara Tuchman,Ballantine Books,8.000000,8.00
4723,Zoids Chaotic Century (Zoids: Chaotic Century ...,Michiro Ueyama,Viz Comics,10.000000,10.00


### 7. For each book in `goodbooks`, compute the list of its authors. Assuming that the number of reviews with a text (column `work_text_reviews_count`) is split equally among all authors, find for each authors the total number of reviews with a text. We will call this quantity the *shared number of reviews with a text*.

In [19]:
# Creiamo una colonna ("List_authors") che contiene la lista degli autori per ogni libro  
goodbooks_books['List_authors'] = goodbooks_books['authors'].str.split(',')
# --------------------------------------------------------------------------------------------------------------- #
# Creiamo una colonna ("Authors_count") che conta quanti autori ci sono per ogni libro

def conta_autori(lista_autori):
    return len(lista_autori)

goodbooks_books['Authors_count'] = goodbooks_books['List_authors'].apply(conta_autori)
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo la lista degli autori per ogni libro che abbiamo calcolato, togliendo le colonne superflue
goodbooks_books[['title','List_authors','Authors_count']]

,title,List_authors,Authors_count
0,"The Hunger Games (The Hunger Games, #1)",[Suzanne Collins],1
1,Harry Potter and the Sorcerer's Stone (Harry P...,"[J.K. Rowling, Mary GrandPrÃ©]",2
2,"Twilight (Twilight, #1)",[Stephenie Meyer],1
3,To Kill a Mockingbird,[Harper Lee],1
4,The Great Gatsby,[F. Scott Fitzgerald],1
...,...,...,...
94,The Picture of Dorian Gray,"[Oscar Wilde, Jeffrey Eugenides]",2
95,"Fifty Shades Freed (Fifty Shades, #3)",[E.L. James],1
96,Dracula,"[Bram Stoker, Nina Auerbach, David J. Skal]",3
97,"The Girl Who Played with Fire (Millennium, #2)","[Stieg Larsson, Reg Keeland]",2


In [20]:
# Creiamo una colonna ("Reviews_per_author") dove, per ogni libro, viene calcolata la quantità di reviews per ogni autore
goodbooks_books['Reviews_per_author'] = goodbooks_books['work_text_reviews_count'] / goodbooks_books['Authors_count']
# --------------------------------------------------------------------------------------------------------------- #
# "Esplodiamo" ogni riga della tabella mettendo su una riga diversa ogni autore delle varie liste
exploded_goodbooks_books = goodbooks_books.explode('List_authors')
# --------------------------------------------------------------------------------------------------------------- #
# Puliamo la tabella dalle colonne superflue
shared_reviews_goodbooks_books = exploded_goodbooks_books[['List_authors','Reviews_per_author']]
# --------------------------------------------------------------------------------------------------------------- #
# Sommiamo le reviews per ogni autore
shared_reviews_authors = shared_reviews_goodbooks_books.groupby('List_authors', as_index = False)['Reviews_per_author'].sum()
# --------------------------------------------------------------------------------------------------------------- #
# Rinominiamo le colonne per dar loro un aspetto più presentabile
shared_reviews_authors = shared_reviews_authors.rename(columns={'List_authors':'Author', 'Reviews_per_author':'shared_number_of_reviews_with_a_text'})
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato
shared_reviews_authors

,Author,shared_number_of_reviews_with_a_text
0,Alan R. Clarke,27890.500000
1,B.M. Mooyaart-Doubleday,6941.666667
2,Bernard Knox,1620.200000
3,CelÃ¢l Ãster,15172.666667
4,David J. Skal,5754.333333
...,...,...
105,Veronica Roth,156896.000000
106,William Golding,26886.000000
107,William Goldman,15630.000000
108,William Shakespeare,7389.000000


### 8. For each year of publication, determine the author that has the largest value of the shared number of reviews with a text.

In [21]:
# Vediamo, per ogni anno, quali sono le shared_number_of_reviews_with_a_text di ogni autoreo
yearly_reviws_per_author = exploded_goodbooks_books.groupby(['original_publication_year','authors','List_authors'], as_index = False)['Reviews_per_author'].sum()
# --------------------------------------------------------------------------------------------------------------- #
# Selezioniamo gli autori (anche a pari merito) con le maggiori shared_number_of_reviews_with_a_text per anno
max_reviews_each_year = yearly_reviws_per_author.loc[yearly_reviws_per_author.groupby('original_publication_year')['Reviews_per_author'].idxmax()]
# --------------------------------------------------------------------------------------------------------------- #
# Ripristiniamo l'indice
max_reviews_each_year.reset_index(drop = True, inplace = True)
# --------------------------------------------------------------------------------------------------------------- #
# Ritocchiamo la colonna degli anni creandone un'altra - più gradevole (formato stringa):
def perfeziona_anno(anno_float):
    if float(anno_float) < 0:
        return str(anno_float)[1:-2] +' B.C.'
    else:
        return str(anno_float)[:-2] +' A.D.'
    
max_reviews_each_year['Publication_year'] = max_reviews_each_year['original_publication_year'].apply(perfeziona_anno)
# --------------------------------------------------------------------------------------------------------------- #
# Rinominiamo le colonne
max_reviews_each_year.rename(columns = {'authors':'Best_reviewed_authors',
    'Reviews_per_author':'Review_that_year_for_author'}, inplace = True)
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo solo le colonne rilevanti
best_yearly_reviewed_authors = max_reviews_each_year[['Publication_year','Best_reviewed_authors','Review_that_year_for_author']]
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato
best_yearly_reviewed_authors

,Publication_year,Best_reviewed_authors,Review_that_year_for_author
0,720 B.C.,"Homer, Robert Fagles, E.V. Rieu, FrÃ©dÃ©ric Mu...",1620.200000
1,1595 A.D.,"William Shakespeare, Robert Jackson",7389.000000
2,1811 A.D.,"Jane Austen, Tony Tanner, Ros Ballaster",3842.000000
3,1813 A.D.,Jane Austen,49152.000000
4,1818 A.D.,"Mary Wollstonecraft Shelley, Percy Bysshe Shel...",6664.333333
5,1847 A.D.,"Charlotte BrontÃ«, Michael Mason",15606.000000
6,1859 A.D.,"Charles Dickens, Richard Maxwell, Hablot Knigh...",4364.333333
7,1868 A.D.,Louisa May Alcott,17090.000000
8,1884 A.D.,"Mark Twain, John Seelye, Guy Cardwell",4149.333333
9,1891 A.D.,"Oscar Wilde, Jeffrey Eugenides",9823.500000


### 9. Assuming that there are no errors in the ISBN fields, find the books in both datasets, and compute the difference of average rating according to the ratings and the goodratings datasets

In [22]:
# MI ASSICURO CHE I LIBRI IN "BOOKS" NON VENGANO PRESI SOLO PER ISBN, MA TUTTI GLI ISBN CORRISPONDENTI
# (in "books" ci sono più ISBN che identificano lo stesso libro)

# Troviamo i libri in comune tra i due dataframe (tenendo solo le colonne rilevanti di entrambi)
books_in_both = pd.merge(books, goodbooks_books, left_on = 'ISBN', right_on = 'isbn')[['Book-Title','Book-Author','Publisher','authors','title','average_rating']]
# --------------------------------------------------------------------------------------------------------------- #
# Selezioniamo solo le colonne che identificano quei libri (+ il rating di goodbooks)
identify_books = books_in_both[['Book-Title', 'Book-Author']]
# --------------------------------------------------------------------------------------------------------------- #
# Facciamo la merge tra questo df e books, per trovare tutti gli ISBN collegati a questi libri
ISBN_identified_books = pd.merge(identify_books, books, on = ['Book-Title', 'Book-Author'])[['Book-Title','Book-Author','ISBN','Publisher']]
# --------------------------------------------------------------------------------------------------------------- #
# Facciamo la LEFT JOIN tra questo dataframe e books_ratings così da trovare i ratings associati a questi ISBN
rating_identified_ISBNs = pd.merge(ISBN_identified_books, counting_book_ratings, left_on = 'ISBN', right_on = 'Cleaned_ISBN', how = 'left')
# --------------------------------------------------------------------------------------------------------------- #
# Facciamo la media dei rating di BOOKS per ogni COPPIA di ['Book-Title','Book-Author']
AVG_rating_identified_ISBNs = rating_identified_ISBNs.groupby(['Book-Title','Book-Author'], as_index = False)['Book-Rating'].mean()
# --------------------------------------------------------------------------------------------------------------- #
# Dimezziamo il valore del rating del dataframe "Books", così da renderlo confrontabile con quello di "Goodbooks"
AVG_rating_identified_ISBNs['Book-Rating'] = AVG_rating_identified_ISBNs['Book-Rating'] / 2

# ADESSO CHE ABBIAMO FATTO LA MEDIA DEL DATAFRAME "BOOKS", LA AGGIUNGIAMO AL DATAFRAME INIZIALE DOVE C'ERA "GOODBOOKS"

# Colleghiamo quest'ultimo df al primo così da aggiungere anche questi rating 
AVG_rating_in_both = pd.merge(AVG_rating_identified_ISBNs, books_in_both, on = ['Book-Title', 'Book-Author'])[['Book-Title','Book-Author','Book-Rating','authors','title','average_rating']]
# --------------------------------------------------------------------------------------------------------------- #
# Aggiungiamo la colonna per le differenze rinomino le colonne
AVG_rating_in_both['Difference_AVG_rating'] = AVG_rating_in_both['Book-Rating'] - AVG_rating_in_both['average_rating']
AVG_rating_compared = AVG_rating_in_both.rename(columns ={'Book-Title':'Title(df_BOOK)','Book-Author':'Authors(df_BOOK)','Book-Rating':'AVG_Rating(df_BOOK)','authors':'Title(df_GOODBOOK)','title':'Authors(df_GOODBOOK)','average_rating':'AVG_Rating(df_GOODBOOK)'})
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato
AVG_rating_compared

,Title(df_BOOK),Authors(df_BOOK),AVG_Rating(df_BOOK),Title(df_GOODBOOK),Authors(df_GOODBOOK),AVG_Rating(df_GOODBOOK),Difference_AVG_rating
0,Bridget Jones's Diary,Helen Fielding,3.806579,Helen Fielding,"Bridget Jones's Diary (Bridget Jones, #1)",3.75,0.056579
1,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling,4.416667,"J.K. Rowling, Mary GrandPrÃ©, Rufus Beck",Harry Potter and the Prisoner of Azkaban (Harr...,4.53,-0.113333
2,The Curious Incident of the Dog in the Night-T...,Mark Haddon,4.294118,Mark Haddon,The Curious Incident of the Dog in the Night-Time,3.85,0.444118
3,The Kite Runner,Khaled Hosseini,4.500000,Khaled Hosseini,The Kite Runner,4.26,0.240000
4,The Outsiders (Now in Speak!),S. E. Hinton,4.049020,S.E. Hinton,The Outsiders,4.06,-0.010980


### 10. Split the users dataset according to the age. One dataset contains the users with unknown age, one with age 0-14, one with age 15-24, one with age 25-34, and so on.

In [23]:
# Convertiamo prima tutte le età in "float" (lo facciamo per sicurezza, in teoria sarebbe già una colonna dtype 'float64'
renewed_users = users['Age'].astype(float)
# --------------------------------------------------------------------------------------------------------------- #
# Ordiniamo il dataframe per età
users_sorted = users.sort_values('Age')
# --------------------------------------------------------------------------------------------------------------- #
# Resettiamo l'indice implicito (in modo che segua lo stesso ordine)
resetted_users_sorted = users_sorted.reset_index(drop = True)
# --------------------------------------------------------------------------------------------------------------- #
# Creo tutti i vari dataframes richiesti:
users_0_14 = resetted_users_sorted[(resetted_users_sorted['Age']>=0) & (resetted_users_sorted['Age']<15)]
users_15_24 = resetted_users_sorted[(resetted_users_sorted['Age']>=15) & (resetted_users_sorted['Age']<25)]
users_25_34 = resetted_users_sorted[(resetted_users_sorted['Age']>=25) & (resetted_users_sorted['Age']<35)]
users_35_44 = resetted_users_sorted[(resetted_users_sorted['Age']>=35) & (resetted_users_sorted['Age']<45)]
users_45_54 = resetted_users_sorted[(resetted_users_sorted['Age']>=45) & (resetted_users_sorted['Age']<55)]
users_55_64 = resetted_users_sorted[(resetted_users_sorted['Age']>=55) & (resetted_users_sorted['Age']<65)]
users_65_74 = resetted_users_sorted[(resetted_users_sorted['Age']>=65) & (resetted_users_sorted['Age']<75)]
users_75_84 = resetted_users_sorted[(resetted_users_sorted['Age']>=75) & (resetted_users_sorted['Age']<85)]
users_85_94 = resetted_users_sorted[(resetted_users_sorted['Age']>=85) & (resetted_users_sorted['Age']<95)]

users_over_94_or_wrong_age = resetted_users_sorted[resetted_users_sorted['Age']>=95]

# N.B: il dataframe contiene utenti con età dichiarata ben al di sopra dei 100 anni. Questi sono stati "accorpati",
# perché, per quanto la loro età non sia per forza "sconosciuta", non sembra neppure realistica - ma mai dire mai.

# N.B.2: Troviamo il database "users_unknown_age" con i valori Null "per differenza"
age_condition = resetted_users_sorted['Age']>=0
aged_users = resetted_users_sorted[age_condition]
users_unknown_age = resetted_users_sorted[~age_condition]
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato
print('# --------------------------------------------------------------------------------------------------------------- #')
print(users_0_14)
print('# --------------------------------------------------------------------------------------------------------------- #')
print()
print('# --------------------------------------------------------------------------------------------------------------- #')
print(users_15_24)
print('# --------------------------------------------------------------------------------------------------------------- #')
print()
print('# --------------------------------------------------------------------------------------------------------------- #')
print(users_25_34)
print('# --------------------------------------------------------------------------------------------------------------- #')
print()
print('# --------------------------------------------------------------------------------------------------------------- #')
print(users_35_44)
print('# --------------------------------------------------------------------------------------------------------------- #')
print()
print('# --------------------------------------------------------------------------------------------------------------- #')
print(users_45_54)
print('# --------------------------------------------------------------------------------------------------------------- #')
print()
print('# --------------------------------------------------------------------------------------------------------------- #')
print(users_55_64)
print('# --------------------------------------------------------------------------------------------------------------- #')
print()
print('# --------------------------------------------------------------------------------------------------------------- #')
print(users_65_74)
print('# --------------------------------------------------------------------------------------------------------------- #')
print()
print('# --------------------------------------------------------------------------------------------------------------- #')
print(users_75_84)
print('# --------------------------------------------------------------------------------------------------------------- #')
print()
print('# --------------------------------------------------------------------------------------------------------------- #')
print(users_85_94)
print('# --------------------------------------------------------------------------------------------------------------- #')
print()
print('# --------------------------------------------------------------------------------------------------------------- #')
print(users_over_94_or_wrong_age)
print('# --------------------------------------------------------------------------------------------------------------- #')
print()
print('# --------------------------------------------------------------------------------------------------------------- #')
print(users_unknown_age)
print('# --------------------------------------------------------------------------------------------------------------- #')

# --------------------------------------------------------------------------------------------------------------- #
      User-ID               City            Region    Country   Age
0      162240            buffalo          new york        usa   0.0
1      130116  lower templestowe          victoria  australia   0.0
2      135954     powder springs           georgia        usa   0.0
3      207491           manassas          virginia        usa   0.0
4      189925      san francisco        california        usa   0.0
...       ...                ...               ...        ...   ...
4308   233813                n/a  british columbia     canada  14.0
4309   181320        pierrefonds            quebec     canada  14.0
4310   132977             sydney               nsw  australia  14.0
4311    73412     belo horizonte      minas gerais     brazil  14.0
4312   181310             austin             texas        usa  14.0

[4313 rows x 5 columns]
# -----------------------------------------

### 11. Find the books that appear only in the goodbooks datasets.

In [24]:
# Facciamo una LEFT JOIN tra il dataframe GOODBOOKS e BOOKS
first_join = pd.merge(goodbooks_books, books, left_on = 'Cleaned_isbn', right_on = 'Cleaned_ISBN', how = 'left')
# --------------------------------------------------------------------------------------------------------------- #
# Ora facciamo la stessa operazione unendo con LEFT JOIN il dataframe ottenuto e il dataframe dei RATINGS
second_join = pd.merge(first_join, revised_book_ratings, left_on = 'Cleaned_isbn', right_on = 'Cleaned_ISBN', how = 'left')
# --------------------------------------------------------------------------------------------------------------- #
# Eliminiamo tutti i libri trovati grazie alla prima unione
third_df = second_join[second_join['Cleaned_ISBN_x'].isnull()]
# --------------------------------------------------------------------------------------------------------------- #
# Eliminiamo anceh quelli trovati con la seconda unione
forth_df = third_df[third_df['Cleaned_ISBN_y'].isnull()]
# --------------------------------------------------------------------------------------------------------------- #
# Eliminiamo tutte le colonne superflue e ripristiniamo l'indice
ultimate_df = forth_df[['authors', 'title']]
ultimate_df = ultimate_df.reset_index(drop = True)
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato
ultimate_df

,authors,title
0,Suzanne Collins,"The Hunger Games (The Hunger Games, #1)"
1,"J.K. Rowling, Mary GrandPrÃ©",Harry Potter and the Sorcerer's Stone (Harry P...
2,Stephenie Meyer,"Twilight (Twilight, #1)"
3,Harper Lee,To Kill a Mockingbird
4,F. Scott Fitzgerald,The Great Gatsby
5,John Green,The Fault in Our Stars
6,Dan Brown,"Angels & Demons (Robert Langdon, #1)"
7,Veronica Roth,"Divergent (Divergent, #1)"
8,"Stieg Larsson, Reg Keeland","The Girl with the Dragon Tattoo (Millennium, #1)"
9,Suzanne Collins,"Catching Fire (The Hunger Games, #2)"


### 12. Assuming that each pair (author, title) identifies a book, for each book find the number of times it appears in the books dataset. Which books appear the most times?

In [25]:
# Facciamo una groupby per Titolo e Autore, e contiamo quante volte questi appaiono
appearence = books.groupby(['Book-Title', 'Book-Author'], as_index = False)['ISBN'].count()
# --------------------------------------------------------------------------------------------------------------- #
# Mettiamo i risultati più alti in testa al dataframe
sorted_appearence = appearence.sort_values('ISBN', ascending = False)
# --------------------------------------------------------------------------------------------------------------- #
# Prendiamo solo le prime tre colonne del dataframe precedente
appearence_times = sorted_appearence.iloc[:, :3]
# --------------------------------------------------------------------------------------------------------------- #
# Rinominiamo l'ultima colonna, altrimenti non significa nulla
appearence_times = appearence_times.rename(columns = {'ISBN': 'Times_it_appears'})
# --------------------------------------------------------------------------------------------------------------- #
# Ripristiniamo l'indice implicito
appearence_times = appearence_times.reset_index(drop=True) 
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato
appearence_times

,Book-Title,Book-Author,Times_it_appears
0,Little Women,Louisa May Alcott,21
1,Wuthering Heights,Emily Bronte,20
2,Adventures of Huckleberry Finn,Mark Twain,20
3,Pride and Prejudice,Jane Austen,18
4,The Secret Garden,Frances Hodgson Burnett,16
...,...,...,...
251178,Heritage of Hastur,Marion Zimmer Bradley,1
251179,Heritage of Music (4 Volume Set),Michael Raeburn,1
251180,Heritage of Shame,Meg Hutchinson,1
251181,Heritage of Stars,Clifford D. Simak,1


In [26]:
# Facciamo una slice del dataframe precedente, tenendo solo la prima riga
most_present = appearence_times.iloc[:1,:]
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato
most_present

,Book-Title,Book-Author,Times_it_appears
0,Little Women,Louisa May Alcott,21


### 13. Find the author with the highest average rating according to the goodbooks datasets.

In [27]:
# Creiamo una colonna che contiene la lista degli autori per ogni libro
goodbooks_books['List_authors'] = goodbooks_books['authors'].str.split(',') 
# --------------------------------------------------------------------------------------------------------------- #
# "Esplodiamo" ogni riga della tabella mettendo su una riga diversa ogni autore delle varie liste:
exploded_goodbooks = goodbooks_books.explode('List_authors')
# --------------------------------------------------------------------------------------------------------------- #
# Creiamo una colonna con l'avg rating moltiplicato per il numero di rating per-ogni-autore&libro
exploded_goodbooks['mul_rating_for_that_book'] = exploded_goodbooks['work_ratings_count'] * exploded_goodbooks['Calculated_average_rating']
# --------------------------------------------------------------------------------------------------------------- #
# Facciamo la groupby per-ogni-autore con la sum()
mul_rating_for_author = exploded_goodbooks.groupby('List_authors', as_index = False)[['work_ratings_count','mul_rating_for_that_book']].sum()
# --------------------------------------------------------------------------------------------------------------- #
# Troviamo l'avg dividendo la somma della colonna creata per il totole rating per-ogni-autore
mul_rating_for_author['avg_rating_for_author'] = mul_rating_for_author['mul_rating_for_that_book'] / mul_rating_for_author['work_ratings_count']
# --------------------------------------------------------------------------------------------------------------- #
# Eliminiamo le colonne superflue
avg_rating_for_author = mul_rating_for_author[['List_authors','avg_rating_for_author']]
# --------------------------------------------------------------------------------------------------------------- #
# Ripristiamo l'indice implicito
avg_rating_for_author.reset_index(drop = True, inplace = True)
# --------------------------------------------------------------------------------------------------------------- #
# Seleziono solo la prima riga
best_rating_author = avg_rating_for_author.iloc[avg_rating_for_author['avg_rating_for_author'].idxmax()]
# --------------------------------------------------------------------------------------------------------------- #
# Mostro il risultato
best_rating_author

List_authors              Rufus Beck
avg_rating_for_author       4.527866
Name: 32, dtype: object

## VERSIONE ALTERNATIVA - QUESITO N. 12

In [28]:
# Proprio perché è facile che nel dataset "BOOKS" compaiano titoli o autori identici ma scritti in modo diverso,
# si propone questa versione di pulizia titoli-autori, realizzata senza particolari moduli o pacchetti ma solo
# usando un po' di "buonsenso" e con i semplici strumenti offerti da Python

# --------------------------------------------------------------------------------------------------------------- #

# Definiamo una funzione di "pulizia" del nome dei libri e degli autori

def crop_title(passed_title):
    cropped_title = ''
    for letter in str(passed_title): # eliminiamo tutto tranne lettere e numeri
        if letter == '(': # eliminiamo i sottotitoli tra parentesi
            break
        if re.search('\w', letter):
            cropped_title += str(letter)
    cropped_title = cropped_title.lower() # eliminiamo le lettere maiuscole
    return cropped_title
# --------------------------------------------------------------------------------------------------------------- #
# Creiamo una colonna con i titoli modificati
books['cropped_title'] = books['Book-Title'].apply(crop_title)
# --------------------------------------------------------------------------------------------------------------- #
# Creiamo una colonna con gli autori modificati
books['cropped_author'] = books['Book-Author'].apply(crop_title)
# --------------------------------------------------------------------------------------------------------------- #
# Troviamo la lista delle coppie titolo-autore che compaiono più spesso
cropped_books = books.groupby(['cropped_title', 'cropped_author'], as_index = False)['ISBN'].count()
# --------------------------------------------------------------------------------------------------------------- #
# Facciamo una merge con "Books" per riprendere anche i titoli originali
selected_books = pd.merge(cropped_books, books, on = ['cropped_title', 'cropped_author'])[['Book-Title','Book-Author','cropped_title','cropped_author','ISBN_x']]
# --------------------------------------------------------------------------------------------------------------- #
# Selezioniamo solo una volta le varie coppie titolo-autore
ultimate_selection = selected_books.loc[selected_books.groupby(['cropped_title', 'cropped_author'])['ISBN_x'].idxmax()][['Book-Title','Book-Author','ISBN_x']]
# --------------------------------------------------------------------------------------------------------------- #
# Mettiamo i risultati più alti in testa al dataframe
ultimate_selection = ultimate_selection.sort_values('ISBN_x', ascending = False)
# --------------------------------------------------------------------------------------------------------------- #
# Rinominiamo l'ultima colonna, altrimenti non significa nulla
ultimate_selection = ultimate_selection.rename(columns = {'ISBN_x': 'Times_it_appears'})
# --------------------------------------------------------------------------------------------------------------- #
# Ripristiniamo l'indice implicito
ultimate_selection = ultimate_selection.reset_index(drop=True) 
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato
ultimate_selection

,Book-Title,Book-Author,Times_it_appears
0,Little Women (Signet Classic),Louisa May Alcott,47
1,Wuthering Heights,EMILY BRONTE,46
2,Pride and Prejudice,Jane Austen,43
3,Jane Eyre,Charlotte Bronte,42
4,Dracula,Bram Stoker,42
...,...,...,...
235392,Hollywood Heroes,Caine Hamilton,1
235393,Hollywood Hex: An Illustrated History of Curse...,Mikita Brottman,1
235394,HOLLYWOOD HORROR (NANCY DREW HARDY BOY SUPERMY...,Carolyn Keene,1
235395,Hollywood Hulk Hogan,Hulk Hogan,1


In [29]:
# Facciamo una slice del dataframe precedente, tenendo solo la prima riga
the_most_appeared = ultimate_selection.iloc[:1,:]
# --------------------------------------------------------------------------------------------------------------- #
# Mostriamo il risultato
the_most_appeared

,Book-Title,Book-Author,Times_it_appears
0,Little Women (Signet Classic),Louisa May Alcott,47
